In [21]:
# OPENAI_API_KEY
import os
from dotenv import load_dotenv

def import_env_var(var_name):
    load_dotenv('./../.env')
    env_var = os.getenv(var_name)
    if env_var is None:
        raise ValueError(f"{var_name} not found. Please check your .env file.")
    else:
        return env_var

OPENAI_API_KEY = import_env_var("OPENAI_API_KEY")

In [22]:
# import json

import sqlite3

# def read_json_file(filepath):
#     with open(filepath, 'r') as file:
#         data = json.load(file)
#     return data

In [23]:
''' For converting JSON to sqlite DB '''
# import sqlite3
# import json

# # Connect to SQLite database
# conn = sqlite3.connect('crewdata.db')
# c = conn.cursor()

# # Create table
# c.execute('''
#     CREATE TABLE crewdata
#     (name text, userid text, crewType text, roleJobTitle text, services text, 
#     tags text, expertise text, yoe integer, minRatePerDay integer, maxRatePerDay integer, location text)
# ''')

# # Load JSON data
# with open('demo_data/crewdata.json') as f:
#     data = json.load(f)

# # Insert JSON data into the table
# for row in data:
#     name = row['name']
#     userid = row['userid']
#     crewType = row['crewType']
#     roleJobTitle = row['roleJobTitle']
#     services = ', '.join(row['services'])
#     tags = ', '.join(row['tags'])
#     expertise = ', '.join(row['expertise'])
#     yoe = row['yoe']
#     minRatePerDay = row['minRatePerDay']
#     maxRatePerDay = row['maxRatePerDay']
#     location = row['location']
    
#     c.execute('''
#         INSERT INTO crewdata VALUES (?,?,?,?,?,?,?,?,?,?,?)
#     ''', (name, userid, crewType, roleJobTitle, services, tags, expertise, yoe, minRatePerDay, maxRatePerDay, location))

# # Commit the transaction
# conn.commit()

# # Close the connection
# conn.close()

' For converting JSON to sqlite DB '

In [24]:
# !pip install langchain-core > NULL
# !pip install langchain-openai > NULL
# !pip install langgraph langchain > NULL
# !python.exe -m pip install --upgrade pip > NULL

In [25]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Set your OpenAI API key
api_key = OPENAI_API_KEY

# # Initialize the ChatOpenAI instance
llm = ChatOpenAI(model="gpt-4o", temperature=0.2, api_key=api_key)
llm_json = ChatOpenAI(model="gpt-4o", temperature=0.2, api_key=api_key).bind(response_format={"type": "json_object"})
from pprint import pprint

import json

In [26]:
def filter_crew_members(roleJobTitle, location, db):
    dbname = db.split('.')[0]
    conn = sqlite3.connect(db)
    c = conn.cursor()
    c.execute(f"SELECT * FROM \'{dbname}\' WHERE roleJobTitle = \'{roleJobTitle}\' AND location = \'{location}\'")
    filtered_data = c.fetchall()
    conn.close()
    return filtered_data

In [27]:
print(filter_crew_members('Camera Operator', 'Dubai', 'crewdata.db'))

[('Omar Al-Mahmoud', 'omar.almahmoud@gmail.com', 'Film Crew', 'Camera Operator', 'Camera Operation, Frame Composition', 'Film, Camera, Composition', 'Camera Operation, Frame Composition, Technical Skills', 7, 500, 750, 'Dubai'), ('Rashid Al-Habsi', 'rashid.alhabsi@gmail.com', 'Film Crew', 'Camera Operator', 'Camera Operation, Focus Pulling', 'Film, Camera, Focus', 'Focus Pulling, Camera Operation, Technical Skills', 8, 520, 770, 'Dubai'), ('Yousef Al-Nassir', 'yousef.alnassir@gmail.com', 'Film Crew', 'Camera Operator', 'Steadicam Operation, Camera Movement', 'Film, Steadicam, Movement', 'Steadicam Operation, Camera Movement, Technical Skills', 6, 540, 790, 'Dubai'), ('Sameer Sharma', 'sameer.sharma@gmail.com', 'Film Crew', 'Camera Operator', 'Camera Operation, Lens Selection', 'Film, Camera, Lens', 'Lens Selection, Camera Operation, Technical Skills', 7, 500, 750, 'Dubai'), ('Rahul Verma', 'rahul.verma@gmail.com', 'Film Crew', 'Camera Operator', 'Camera Operation, Frame Composition', '

In [28]:
'''
PLAN :
    expand project details
    break into queries
    check if all queries satisfied
    check if all queries are valid
    call to DB for all qualified enteries
'''

'\nPLAN :\n    expand project details\n    break into queries\n    check if all queries satisfied\n    check if all queries are valid\n    call to DB for all qualified enteries\n'

In [29]:

def get_detailed_desc(desc):
    prompt_detailed_desc = "You are the production advisor and primary contract for the customer. The customer will come to you and describe their project. You need to make the necessary changes so that project details are completely described and even new members can understand the need. It should be such that everyone can get the overview of the project at a glance. The description should try to contain the following in a paragraph : a descriptive overview of the project, locations that will be involved and task at each location, and whether the same team will travel to different locations or if customer will require different teams at different locations. No formatting should be applied to the output, for better clearity and understanding."
    messages = [
        ("system", prompt_detailed_desc),
        ("user", f"This is the project details from user : {desc}"),
    ]
    response = llm.invoke(messages)
    detailed_desc = response.content
    return detailed_desc


def get_unique_roles(db):
    conn = sqlite3.connect(db)
    c = conn.cursor()
    query = "SELECT DISTINCT roleJobTitle FROM crewdata"
    c.execute(query)
    rows = c.fetchall()
    roleJobTitles = [row[0] for row in rows]
    conn.close()

    return roleJobTitles


def get_crew_requirements(detailed_desc, roleJobTitle):
    prompt_crew_requirement_getter = f"You are an experienced film production assistant and an expert in planning and organizing film crews. Your task is to provide comprehensive list of crew members required to complete a film production project based on the details provided by the user. This includes identifying all essential crew roles, detailing their primary responsibilities, and specifying the number of individuals needed for each specific role. Also, consider any specialized roles required for the specific requirements of the project. Make sure that all roles chose must only be from these : {roleJobTitle}. Note that if project is to be done in multiple locations we might need crew at multiple location or we might travel, understand the requirement and give output accordingly. Now if two camera operator are required one at location1 and other at location2 then output them separately like one camera operator at location1 and one camera operator at locatin2.  Output must be in JSON format and should contain only following fields :[roleJobTitle, responsibilty_in_project, number_needed, location]"
    messages = [
        ("system", prompt_crew_requirement_getter),
        ("user", f"This is the project details from user : {detailed_desc}"),
    ]
    response = llm_json.invoke(messages)
    crew_requirements = json.loads(response.content)
    try : 
        crew_requirements = crew_requirements["crew"]
    except : 
        crew_requirements = crew_requirements
    return crew_requirements


def get_queries(crew_requirements):
    prompt_query_generator = "You are the database manager at my film production management company, a client will contact you with the crew requirement and then you need to define queries to database to get the data for that particular role. Queries should be for a SQL DB and should filter data out based on, role and location. Output should be in JSON format as list of query. You are a man of words your responses are upto the point and no unnecessary blaberring and words are mentioned, only JSON is outputted"
    messages = [
        ("system", prompt_query_generator),
        ("user", f"Following is the requirement of the customer : {crew_requirements}"),
    ]
    response = llm_json.invoke(messages)
    queries = json.loads(response.content)
    
    try : 
        queries = queries["queries"]
    except : 
        queries = queries

    return queries


def get_selected_crews(filtered_crew, number_needed, hiring_role, detailed_desc):
    prompt_crew_selection = "You are a HR in my firm and you have to select the best possible crew. Your subordinates will provide you with the project detail, the crew that match the criteria for that particular job title, and the number of the crew we need to hire for that project. Now you need to select the best possible crew for that particular role and the reason why you preferred that particular person. Remember, year of experience should not be the only factor. Make sure preferred member can work well in the project. Output should be in JSON format which should follow this: {'UserId': , 'Preferred_because'}. Only output JSON. You don't need to put anything extra as your output will be directly fed to a function, so just output JSON."
    # selected_crews = []
    messages = [
        ("system", prompt_crew_selection),
        ("user", f"Following is the list of available members for the project: {filtered_crew}, you need to hire {number_needed} members, for the role of {hiring_role} for this project: {detailed_desc}"),
    ]

    response = llm_json.invoke(messages)
    selected_crews = json.loads(response.content)
    # selected_crews.append(selected_crew)
    return selected_crews

In [30]:
from langgraph.graph import END, StateGraph
from typing import TypedDict, List, Annotated

In [31]:
class State(TypedDict):
    num_steps : int
    project_detail_from_customer : str
    detailed_desc : str
    roleJobTitles : List[str]
    crew_requirements : List[dict]
    queries : List[str]
    selected_crews : List[dict]

In [32]:
# @NODES
# description of the project
def detailed_desc_getter(State):
    num_steps = int(State['num_steps'])
    num_steps += 1

    project_detail_from_customer = State["project_detail_from_customer"]
    print("\n ########################################################################################################################## \n")
    print("------------------DETAILED DESCRIPTION GETTER----------------")
    pprint("project_detail_from_customer:")
    pprint(project_detail_from_customer, width=140, indent=10)
    print("num_steps:", num_steps)

    detailed_desc = get_detailed_desc(project_detail_from_customer)
    pprint("detailed_desc:")
    pprint(detailed_desc, width=140, indent=10)
    return {"detailed_desc" : detailed_desc, "num_steps" : num_steps}


# all the available crew roles
def unique_roles_getter(State):
    num_steps = int(State['num_steps'])
    num_steps += 1

    # print("\n ########################################################################################################################## \n")
    # print("------------------UNIQUE ROLES GETTER----------------")
    roleJobTitles = get_unique_roles('crewdata.db')
    print("roleJobTitles:", roleJobTitles)
    return {"roleJobTitles" : roleJobTitles, "num_steps" : num_steps}


# break into crew requirements
def crew_requirement_getter(State):
    num_steps = int(State['num_steps'])
    num_steps += 1
    detailed_desc = State["detailed_desc"]
    roleJobTitles = State["roleJobTitles"]

    print("\n ########################################################################################################################## \n")
    print("------------------CREW REQUIREMENT GETTER----------------")
    crew_requirements = get_crew_requirements(detailed_desc, roleJobTitles)
    pprint("crew_requirements:")
    pprint(crew_requirements, width=140, indent=10)
    return {"crew_requirements" : crew_requirements, "num_steps" : num_steps}


# break into queries
def queries_getter(State):
    num_steps = int(State['num_steps'])
    num_steps += 1
    crew_requirements = State["crew_requirements"]

    # print("\n ########################################################################################################################## \n")
    # print("------------------QUERIES GETTER----------------")
    queries = get_queries(crew_requirements)
    pprint("queries:")
    pprint(queries, width=140, indent=10)
    return {"queries" : queries, "num_steps" : num_steps}


# check if all queries satisfied ----> planning to drop as I am structuring queries myself, for every requirement               
# check if all queries are valid ----> planning to drop as I am structuring queries myself


# call to DB for all qualified enteries
def crew_selection(State):
    num_steps = int(State['num_steps'])
    num_steps += 1

    detailed_desc = State["detailed_desc"]
    crew_requirements = State["crew_requirements"]

    print("\n ########################################################################################################################## \n")
    print("------------------CREW SELECTION----------------")
    selected_crews = []
    for crew in crew_requirements:
        filtered_crew = filter_crew_members(crew["roleJobTitle"], 'Dubai', 'crewdata.db')
        number_needed = crew["number_needed"]
        hiring_role = crew["roleJobTitle"]

        selected_crews_for_task = get_selected_crews(filtered_crew, number_needed, hiring_role, detailed_desc)
        selected_crews.append({hiring_role:selected_crews_for_task})
        pprint("selected_crews_for_task:")
        pprint(selected_crews_for_task, width=140, indent=10)
    
    return {"selected_crews" : selected_crews, "num_steps" : num_steps}

    
def state_printer(state):
    """print the state"""
    # print("\n ########################################################################################################################## \n")
    # print("------------------STATE PRINTER----------------")
    print("num_steps:", state["num_steps"])
    print("project_detail_from_customer:", state["project_detail_from_customer"])
    print("detailed_desc:", state["detailed_desc"])
    print("roleJobTitles:", state["roleJobTitles"])
    print("queries:", state["queries"])
    print("selected_crews:", state["selected_crews"])
    return


In [33]:
workflow = StateGraph(State)

In [34]:
workflow.add_node("detailed_desc_getter", detailed_desc_getter)
workflow.add_node("unique_roles_getter", unique_roles_getter)
workflow.add_node("crew_requirement_getter", crew_requirement_getter)
workflow.add_node("queries_getter", queries_getter)
workflow.add_node("crew_selection", crew_selection)
workflow.add_node("state_printer", state_printer)

workflow.add_edge("detailed_desc_getter", "unique_roles_getter")
workflow.add_edge("unique_roles_getter", "crew_requirement_getter")
workflow.add_edge("crew_requirement_getter", "queries_getter")
workflow.add_edge("queries_getter", "crew_selection")
workflow.add_edge("crew_selection", "state_printer")

In [35]:
workflow.set_entry_point("detailed_desc_getter")
workflow.add_edge("state_printer", END)

In [36]:
app = workflow.compile()

In [37]:
project_detail_from_customer = "A short film to be shot in Delhi with only 2 actors and 1 camera man and 1 director and 1 make up artist, no extra crew needed"
inputs = {"project_detail_from_customer": project_detail_from_customer,"num_steps":0}

# for op in app.stream(inputs):
#     print(op)

# app.stream(inputs)

In [38]:
var = app.invoke(inputs)


 ########################################################################################################################## 

------------------DETAILED DESCRIPTION GETTER----------------
'project_detail_from_customer:'
'A short film to be shot in Delhi with only 2 actors and 1 camera man and 1 director and 1 make up artist, no extra crew needed'
num_steps: 1
'detailed_desc:'
('The project involves the production of a short film to be shot entirely in Delhi. The film will feature only two actors, and the '
 'production team will consist of one cameraman, one director, and one makeup artist. No additional crew members will be required for this '
 'project. All tasks, including filming, directing, and makeup, will be managed by this small, dedicated team. The same team will handle '
 'all aspects of the shoot, ensuring a streamlined and cohesive production process. The entire project will take place in various '
 'locations within Delhi, with the team traveling together to each site as 

In [39]:
var["selected_crews"]

[{'Director': {'UserId': 'ahmed.saeed@gmail.com',
   'Preferred_because': 'Ahmed Saeed has extensive experience in production planning and coordination, which is crucial for managing a small, dedicated team and ensuring a streamlined production process. His skills in visual storytelling will also enhance the quality of the short film.'}},
 {'Camera Operator': {'UserId': 'sameer.sharma@gmail.com',
   'Preferred_because': 'Sameer Sharma has the required skills in Camera Operation and Lens Selection, which are crucial for a small, dedicated team. His experience and technical skills make him a versatile choice for a streamlined production process. Additionally, his cost is within a reasonable range for the project budget.'}},
 {'Makeup Artist and Hair Stylist': {'UserId': 'alisha.sarkar@example.com',
   'Preferred_because': 'Alisha Sarkar has expertise in Special Effects Makeup and Hair Extensions, which can be valuable for a short film production. Her skills in special effects techniques 